In [88]:
import os
import imp
import json
import joblib
import pandas as pd
import JS_crawler as JS
from tqdm import tqdm

In [163]:
# 모듈 reload
imp.reload(JS)

<module 'JS_crawler' from '/Users/jinseok/Jupyter/투빅스/conference/Tobigs-Recommendation-Conf-Phase2/src/data_collect/JS_crawler.py'>

# 01-트리바고 서울 호텔 기준 인근 음식점(네이버 지도)

In [39]:
hotel_seoul = pd.read_excel(os.path.join("..","..", "data","JS_01_Trivago_seoul_hotel.xlsx"))

In [37]:
i = 0
place_list = []

for hotel, lat, lng in tqdm(hotel_seoul[['item_id', 'lat', 'lng']].values):
    value = JS.getPlaces(hotel, lat, lng)
    if value == 'end':
        continue
    
    place_list.extend(value)
    i += 1

100%|██████████| 863/863 [04:48<00:00,  2.99it/s]


In [40]:
places = pd.json_normalize(place_list)

In [49]:
places = places.drop_duplicates('id')

# 02-places에 해당하는 영수증 리뷰

In [ ]:
# 서울 호텔 인근 places는 총 4148개

In [54]:
places = pd.read_excel(os.path.join("..","..", "data","JS_02_places_naverMaps.xlsx"))

In [62]:
i = 0
users = []
for idx in tqdm(places['id']):
    value = JS.getUsers(idx)
    users.extend(value)
    i += 1
    
    if i == 2:
        break

  0%|          | 1/4148 [00:03<3:42:57,  3.23s/it]


In [98]:
place_users = pd.DataFrame(users, columns = ['nick','review', 'star', 'href', 'place_id'])

In [100]:
joblib.dump(place_users, os.path.join("..","..", "data",'JS_03_place_users_list.pkl'))

['../../data/JS_03_place_users_list.pkl']

# 03-각 users의 idno 가져오기

In [102]:
user_list = joblib.load(os.path.join("..","..", "data","JS_03_place_users_list.pkl"))

In [109]:
# 각 유저가 남긴 다른 리뷰를 모두 가져오기 위해서 고유 유저가 몇명인지 확인해야함
# 4148개의 places에 리뷰를 남긴 user는 총 37038명
user_list['href'].nunique()

37038

In [117]:
unique_user = user_list[['nick', 'href']].drop_duplicates('href')

In [118]:
unique_user

,nick,href
0,DreamWalker,5e54ee7a8f87a842bc78e1e1
1,효지니,5c0803580cab5ff50aad602d
2,꾸미,5c15e7756f069e27d36b9aaa
3,ksb******,5b6fc61dedc436284b1a2d62
4,luv****,5dbd0bc18f87a842bc207c32
...,...,...
24301,럭키맘,5d6cd90e8f87a842bc5e2f04
24302,타워버거,5c0669f1021848e953d2ee08
24303,gks******,5de0e4439ec8258e4accb058
24304,고고고,5c0333f50a8688980bfe27c7


In [130]:
url = "https://m.place.naver.com/my/graphql"
headers = {
  'Content-Type': 'application/json'
}

In [133]:
i = 0
users_idno = []

for href in tqdm(unique_user['href']):
    value = JS.getIDno(href)
    if value == 'end':
        print(i)
        break
    
    users_idno.append(value)
    i += 1

  0%|          | 0/37038 [00:00<?, ?it/s]


In [ ]:
users_info = pd.DataFrame(users_idno, columns = 'href', 'idno', 'nick', 'count')

In [ ]:
joblib.dump(users_info, os.path.join("..","..", "data",'JS_04_users_info.pkl'))

# 04-각 user의 모든 리뷰 가져오기

In [154]:
users_info = joblib.load(os.path.join("..","..", "data","JS_04_users_info.pkl"))

In [157]:
users_info = users_info.sort_values('count', ascending = False)
users_info

,nick_x,href,idno,nick_y,count
7664,호호소녀,5c6fc8159b90dc048b43bfa4,6wKan,호호소녀,2796
3149,쪼콜렛,5ba0b88280c03107ddd9ea9c,1LMMA,쪼콜렛,2661
4388,랄라라,5c6e6ad54bb61f68d9a4d398,KALK,랄라라,2604
29089,12,5c4c1eb747bf2c73612d6986,2vzdR,12,2484
8957,마스터피스,5cd22aaf8f87a842bc2c6704,1MruI,마스터피스,2450
...,...,...,...,...,...
36909,꼬리카레,5ce8aaf61d5cf0631ce410cc,6VgCO,꼬리카레,1
7279,김수연,5dea73c39ec8258e4a2b8921,3EMAJ,김수연,1
713,파파야,5be54781d170ccf4e68a8c67,5oQGC,파파야,1
709,DD,5e59f2838f87a842bc3926b3,n0fW,DD,1


In [159]:
users_info[['idno', 'count']].values

array([['6wKan', 2796],
       ['1LMMA', 2661],
       ['KALK', 2604],
       ...,
       ['5oQGC', 1],
       ['n0fW', 1],
       ['zDxj', 1]], dtype=object)

In [165]:
i = 0
reviews = []
breaker = False

for idno, count in tqdm(users_info[['idno', 'count']].values):
    if breaker == True:
        break
    pages = (count // 100) + 1 # 총 리뷰가 100개 이상인 user
#     pages = 1 # 총 리뷰가 100개 미만인 user
    for page in range(1, pages+1):
        value = JS.getRv(idno, str(page))
        if value == 'end':
            print(i)
            breaker = True
            break
        value = list(map(lambda x : JS.addv(x, idno), value))
        reviews.extend(value)
    i += 1
    if i == 2:
        break

  0%|          | 1/37038 [00:21<218:39:05, 21.25s/it]

In [168]:
joblib.dump(reviews, os.path.join("..","..", "data",'JS_05_reviews.pkl'))

['../../data/JS_05_reviews.pkl']